### Dependencies

In [1]:
import os
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split


### Dataset

In [ ]:
class ADDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): DataFrame containing image paths and labels.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get image path and label
        img_path = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]
        
        # Load image
        image = Image.open(img_path).convert("RGB")
        
        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label



transform = transforms.Compose([
    transforms.Resize((176, 176)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize with ImageNet stats
])

### CNN

In [ ]:
class FeatureExtractionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FeatureExtractionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.batch_norm(x)
        x = self.max_pool(x)
        return x


class ADCNN(nn.Module):
    def __init__(self, dropout_rate=0.25):
        super(ADCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Feature Extraction Blocks
        self.block1 = FeatureExtractionBlock(16, 32)  
        self.block2 = FeatureExtractionBlock(32, 64) 
        self.block3 = FeatureExtractionBlock(64, 128)
        self.block4 = FeatureExtractionBlock(128, 256)

        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.dropout2 = nn.Dropout(p=dropout_rate)
        
        # Fully connected layers
        self.fc1 = nn.Linear(256 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 64) 
        self.fc4 = nn.Linear(64, 4)
        
        # Softmax layer is applied in forward
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.dropout1(x)
        x = self.block4(x)
        x = self.dropout2(x)

        x = torch.flatten(x, start_dim=1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        #x = self.softmax(x)
        return x

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = ADCNN().to(device)
model.load_state_dict(torch.load(r'C:\Users\rohan\Desktop\184a\AD-Detection\trained_cnn.pth'))

cuda:0


C:\Users\rohan\AppData\Local\Temp\ipykernel_24760\2429344213.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r'C:\Users\rohan\Desktop\18

<All keys matched successfully>

### Transfer Learning

Dataset: https://www.kaggle.com/datasets/lukechugh/best-alzheimer-mri-dataset-99-accuracy/data?select=Combined+Dataset

Preprocessing

In [ ]:
import cv2
import numpy as np

def align_images(image, reference):
    # Detect features and compute descriptors
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(image, None)
    kp2, des2 = sift.detectAndCompute(reference, None)

    matcher = cv2.BFMatcher()
    matches = matcher.knnMatch(des1, des2, k=2)

    # Filter matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # Get corresponding points
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    matrix, _ = cv2.estimateAffinePartial2D(src_pts, dst_pts)

    aligned_image = cv2.warpAffine(image, matrix, (reference.shape[1], reference.shape[0]))
    return aligned_image

def preprocess_image(image_path, output_path):
    image = cv2.imread(image_path)

    # Step 1: Reorientation
    image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)

    # # Step 2: Registration (using a reference image)
    # reference = cv2.imread(r'C:\Users\rohan\Desktop\184a\AD-Detection\archive\Data\Mild-Dementia\OAS1_0028_MR1_mpr-2_141.jpg')
    # image = align_images(image, reference)

    # Step 3: Skull-stripping
    # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # edges = cv2.Canny(gray, 100, 200)
    # mask = cv2.dilate(edges, None, iterations=2)
    # image = cv2.bitwise_and(image, image, mask=mask)

    # Histogram Equalization
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.equalizeHist(gray)
    cv2.imwrite(output_path, image)

In [6]:
import cv2

def darken_image(image_path, output_path, alpha=0.5, beta=0):

    # Load the image
    image = cv2.imread(image_path)

    # Adjust brightness
    darkened_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

    cv2.imwrite(output_path, darkened_image)

    return darkened_image


In [ ]:
import os

folder = r'C:\Users\rohan\Desktop\184a\AD-Detection\archive2\Combined Dataset\train'
processed_path = r'C:\Users\rohan\Desktop\184a\AD-Detection\archive2\Combined Dataset\processed'
for subfolder in os.listdir(folder):
    subfolder_path = os.path.join(folder, subfolder)
    processed_subpath = os.path.join(processed_path, subfolder)
    for image_filename in os.listdir(subfolder_path):
        image_path = os.path.join(subfolder_path, image_filename)
        processed_image = os.path.join(processed_subpath, image_filename)
        image = Image.open(image_path)
        rotated_image = image.transpose(Image.ROTATE_90)
        rotated_image.save(processed_image)

        preprocess_image(processed_image, processed_image) # Histogram equalization

        darken_image(processed_image, processed_image) # darken image

        
        

Loading and Evaluation

In [ ]:
images2 = []
labels2 = []
label = 0
folder = r'C:\Users\rohan\Desktop\184a\AD-Detection\archive2\Combined Dataset\processed'
for subfolder in os.listdir(folder):
    subfolder_path = os.path.join(folder, subfolder)
    for image_filename in os.listdir(subfolder_path):
        image_path = os.path.join(subfolder_path, image_filename)
        images2.append(image_path)
        labels2.append(label)
    label += 1

df2 = pd.DataFrame({'image': images2, 'label': labels2})
df2

,image,label
0,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
1,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
2,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
3,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
4,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
...,...,...
11514,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,3
11515,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,3
11516,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,3
11517,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,3


In [ ]:
# Create Datasets for training and testing
tl_dataset = ADDataset(dataframe=df2, transform=transform)

print(len(tl_dataset))


# Create DataLoaders
batch_size = 16
tl_dataloader = DataLoader(tl_dataset, batch_size=batch_size, shuffle=True)


# Test the DataLoader
print("Transfer Learning DataLoader:")
for images, labels in tl_dataloader:
    print(images.shape)
    print(labels)
    break

11519
Transfer Learning DataLoader:
torch.Size([16, 3, 176, 176])
tensor([3, 2, 2, 3, 0, 2, 2, 0, 3, 2, 3, 3, 2, 2, 1, 1])


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import itertools
def evaluate_transfer_learning(model, tl_loader, device='cuda'):
    y_pred_list = []
    y_target_list = []

    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for inputs, labels in tl_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, y_pred = torch.max(outputs, 1)
            
            y_pred_list.append(y_pred.cpu().numpy())
            y_target_list.append(labels.cpu().numpy())

    y_pred_list = list(itertools.chain.from_iterable(y_pred_list))
    y_target_list = list(itertools.chain.from_iterable(y_target_list))

    conf_matrix = confusion_matrix(y_target_list, y_pred_list)
    print("Confusion Matrix of the Test Set")
    print("-----------")
    print(conf_matrix)

    precision = precision_score(y_target_list, y_pred_list, average='weighted')
    recall = recall_score(y_target_list, y_pred_list, average='weighted')
    f1 = f1_score(y_target_list, y_pred_list, average='weighted')

    print(f"Precision of the Model :\t{precision:.4f}")
    print(f"Recall of the Model    :\t{recall:.4f}")
    print(f"F1 Score of the Model  :\t{f1:.4f}")

evaluate_transfer_learning(model, tl_dataloader, device='cuda')

Confusion Matrix of the Test Set
-----------
[[ 266    0 2200  273]
 [ 293    0 1541  738]
 [ 162    0 2241  797]
 [ 338    0 1942  728]]
Precision of the Model :	0.2133
Recall of the Model    :	0.2808
F1 Score of the Model  :	0.2138


c:\Users\rohan\Desktop\184a\AD-Detection\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
